<a href="https://colab.research.google.com/github/mendesvi/COMPILADORES/blob/main/P3_AST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terceiro Projeto: Construindo a Árvore de Sintaxe Abstrata

Uma árvore de sintaxe abstrata (do inglês, *Abstract Syntax Tree* - AST) é uma estrutura de dados que representa melhor a estrutura do código do programa do que a árvore de sintaxe. Uma AST pode ser editada e aprimorada com informações como propriedades e anotações para cada elemento que ela contém. Seu objetivo neste terceiro projeto é transformar a árvore de sintaxe em uma AST.

## Nós de uma Árvore de Sintaxe Abstrata

Esta seção define classes para diferentes tipos de nós de uma AST. Durante a análise sintática, você criará esses nós e os conectará. Em geral, você terá um nó AST diferente para cada tipo de regra gramatical. 

A classe `Node` abaixo deve ser usada como base para implementar os diferentes tipos de nós da AST.

```python
class Node(object):
    """
    Base class example for the AST nodes.

    By default, instances of classes have a dictionary for attribute storage.
    This wastes space for objects having very few instance variables.
    The space consumption can become acute when creating large numbers of instances.

    The default can be overridden by defining __slots__ in a class definition.
    The __slots__ declaration takes a sequence of instance variables and reserves
    just enough space in each instance to hold a value for each variable.
    Space is saved because __dict__ is not created for each instance.
    """
    __slots__ = ("coord",)

    def __init__(self, coord=None):
        self.coord = coord

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()
```

Para cada um dos nós AST específicos, você precisa adicionar a especificação `__slots__` apropriada que indica quais campos devem ser armazenados:

```python
class Program(Node):

    __slots__ = ("gdecls",)

    def __init__(self, gdecls, coord=None):
        super().__init__(coord)      
        self.gdecls = gdecls

    def children(self):
        nodelist = []
        for i, child in enumerate(self.gdecls or []):
            nodelist.append(("gdecls[%d]" % i, child))
        return tuple(nodelist)
```

Apenas como outro exemplo, para um operador binário, você pode armazenar o operador, a expressão esquerda e a expressão direita assim:

```python
class BinaryOp(Node):

    __slots__ = ("op", "lvalue", "rvalue",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.lvalue = left
        self.rvalue = right

    def children(self):
        nodelist = []
        if self.lvalue is not None:
            nodelist.append(("lvalue", self.lvalue))
        if self.rvalue is not None:
            nodelist.append(("rvalue", self.rvalue))
        return tuple(nodelist)

    attr_names = ("op",)
```

Para uma constante, você pode armazenar o tipo e o valor, assim:

```python
class Constant(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value")
```

Os campos especificados em `__slots__` que não forem nós filhos retornados no método `children()` , mas sim atributos do próprio nó que serão exibidos quando ele for impresso, devem ter seus nomes indicados em `attr_names`.

**Sugestão:** Você deve começar de forma simples e trabalhar incrementalmente até construir a gramática completa.

## Classes de Nós AST

A lista abaixo define as classes de nós AST e os nomes de atributos esperados que devem ser usados no analisador sintático:

ArrayDecl ( ), ArrayRef ( ), Assert ( ), Assignment (op), BinaryOp (op), Break ( ), Compound ( ), Constant (type, value), Decl (name), DeclList ( ), EmptyStatement ( ), ExprList ( ), For ( ), FuncCall ( ), FuncDecl ( ), FuncDef ( ), GlobalDecl ( ), ID (name), If ( ), InitList ( ), ParamList ( ), Print ( ), Program ( ), Read ( ), Return ( ), Type (name), VarDecl ( ), UnaryOp (op), While ( ).

## Mostrando a AST

Considere como exemplo o seguinte programa em uC:

```cpp
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}
```

A impressão da AST para o exemplo acima é:

```
Program:
    GlobalDecl:
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 2:1
            Constant: int, 3 @ 2:9
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:13
            Decl: ID(name=i)
                VarDecl:
                    Type: int @ 4:3
                ID: j @ 4:11
            Decl: ID(name=k)
                VarDecl:
                    Type: int @ 5:3
                Constant: int, 3 @ 5:11
            Decl: ID(name=p)
                VarDecl:
                    Type: int @ 6:3
                BinaryOp: * @ 6:11
                    Constant: int, 2 @ 6:11
                    ID: j @ 6:15
            Assert: @ 7:3
                BinaryOp: == @ 7:10
                    ID: p @ 7:10
                    BinaryOp: * @ 7:15
                        Constant: int, 2 @ 7:15
                        ID: i @ 7:19
```

Os métodos para gerar uma representação textual dos nós AST e imprimir todos os seus atributos estão mostrados abaixo:

In [ ]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord",)

    def __init__(self, coord=None):
        self.coord = coord

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

## Lidando com informações de linha e coluna na AST

A classe `Coord` abaixo deve ser usada para armazenar (e mostrar na AST) as linhas e colunas das produções no código fonte.

In [ ]:
class Coord:
    """Coordinates of a syntactic element. Consists of:
    - Line number
    - (optional) column number, for the Lexer
    """

    __slots__ = ("line", "column")

    def __init__(self, line, column=None):
        self.line = line
        self.column = column

    def __str__(self):
        if self.line and self.column is not None:
            coord_str = "@ %s:%s" % (self.line, self.column)
        elif self.line:
            coord_str = "@ %s" % (self.line)
        else:
            coord_str = ""
        return coord_str

Para capturar as coordenadas do símbolo terminal mais à esquerda de uma produção, use o seguinte código na classe `UCParser` (a coordenada inclui os números de linha e de coluna e ambos seguem a semântica do analisador léxico, começando em 1):

```python
    def _token_coord(self, p):
        line, column = self.lexer._make_location(p)
        return Coord(line, column)
```

## Aspectos adicionais de implementação

### Construindo declarações

Na linguagem uC, várias variáveis pode ser declaradas em uma mesma linha. Por exemplo:

```cpp
int x, y, z = 5;
```

Porém, na AST, devemos dividí-las para separar os nós `Decl`. O trecho de código abaixo divide as declarações e sempre retorna uma lista de nós `Decl`, mesmo que seja um único elemento.

```python
    def _build_declarations(self, spec, decls):
        """ Builds a list of declarations all sharing the given specifiers.
        """
        declarations = []

        for decl in decls:
            assert decl['decl'] is not None
            declaration = Decl(
                    name=None,
                    type=decl['decl'],
                    init=decl.get('init'),
                    coord=decl['decl'].coord)

            fixed_decl = self._fix_decl_name_type(declaration, spec)
            declarations.append(fixed_decl)

        return declarations
```

Devido à ordem em que os declaradores são construídos, eles precisam ser corrigidos para parecer uma AST normal.

Quando uma declaração chega da construção de sintaxe, ela tem os seguintes problemas:

- O nó `VarDecl` mais interno não tem tipo (porque o tipo básico só é conhecido no nível de declaração mais alto)
- A declaração não tem nome de variável, pois é salva no `VarDecl` mais interno

O método abaixo corrige esses problemas:

```python
    def _fix_decl_name_type(self, decl, typename):
        """ Fixes a declaration. Modifies decl.
        """
        # Reach the underlying basic type
        type = decl
        while not isinstance(type, VarDecl):
            type = type.type

        decl.name = type.declname
        if not typename:
            # Functions default to returning int
            if not isinstance(decl.type, FuncDecl):
                self._parse_error("Missing type in declaration", decl.coord)
            type.type = Type('int', coord=decl.coord)
        else:
            type.type = Type(typename.name, coord=typename.coord)
            
        return decl
```

A AST para a declaração:

```cpp
int x, y, z = 5;
```

Deve se parecer com:

```
Program:
    GlobalDecl:
        Decl: ID(name=x)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=y)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=z)
            VarDecl:
                Type: int @ 1:1
            Constant: int, 5 @ 1:15
```

### Construindo definições de funções

As declarações sempre vêm como listas (porque podem ser várias em uma linha), portanto, uma decisão de projeto foi envolver a definição da função em uma lista também, para tornar o valor de retorno de `global_declaration` homogêneo. Por outro lado, não incorporamos um nó `FuncDef` dentro da classe `GlobalDecl`. Então teremos:

```python
    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return Program(p.global_declaration_list)
        
    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1        

    # <global_declaration> ::= <function_definition>
    #                        | <declaration>
    @_('function_definition',
       'declaration')
    def global_declaration(self, p):
        if hasattr(p, 'function_definition'):
            return p.function_definition
        else:
            return GlobalDecl(p.declaration)
```

No exemplo:

```cpp
int i, j;
int main() {}
```

A seguinte AST é produzida:

```
Program:
    GlobalDecl:
        Decl: ID(name=i)
            VarDecl:
                Type: int @ 1:1
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 1:1
    FuncDef:
        Type: int @ 2:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 2:1
        Compound: @ 2:12
```

### Modificador de tipo

Um tipo em uC consiste em uma declaração de tipo básico, com uma lista de modificadores. Por exemplo:

```cpp
int c[5];
```

A declaração básica aqui é `int c`, e o arranjo é o modificador. Declarações básicas são representadas por `VarDecl` e os modificadores são `FuncDecl` e `ArrayDecl`.

O padrão afirma que sempre que um novo modificador é analisado, ele deve ser adicionado ao final da lista de modificadores. Por exemplo:

__Declaradores de arranjos__

Em uma declaração `T D` onde `D` tem a forma `D1 [constant_expression]` e o tipo do identificador na declaração `T D1` é `type-modifier T`, o tipo do identificador de `D` é `type-modifier array of T`.

Isso é o que o método abaixo faz. O declarador que ele recebe pode ser uma lista de declaradores que terminam com `VarDecl`. Ele adiciona o modificador ao final desta lista, logo antes do `VarDecl`.

```python
    def _type_modify_decl(self, decl, modifier):
        """Tacks a type modifier on a declarator, and returns
           the modified declarator.
           Note: the declarator and modifier may be modified
        """
        modifier_head = modifier
        modifier_tail = modifier

        # The modifier may be a nested list. Reach its tail.
        while modifier_tail.type:
            modifier_tail = modifier_tail.type

        # If the decl is a basic type, just tack the modifier onto it
        if isinstance(decl, VarDecl):
            modifier_tail.type = decl
            return modifier
        else:
            # Otherwise, the decl is a list of modifiers. Reach
            # its tail and splice the modifier onto the tail,
            # pointing to the underlying basic type.
            decl_tail = decl

            while not isinstance(decl_tail.type, VarDecl):
                decl_tail = decl_tail.type

            modifier_tail.type = decl_tail.type
            decl_tail.type = modifier_head
            return decl
```

A AST para a declaração:

```cpp
int c[5];
```

Deve se parecer com:

```
Program:
    GlobalDecl:
        Decl: ID(name=c)
            ArrayDecl:
                VarDecl:
                    Type: int @ 1:1
                Constant: int, 5 @ 1:7
```

### Expressões únicas

Expressões na gramática uC podem ser vistas como listas, mas também podem aparecer como uma única expressão. No último caso, para não "poluir" a AST, uma decisão de projeto foi tratar a expressão única de maneira diferente. 

Por exemplo, para as instruçãos de impressão abaixo:

```cpp
char name[] = "Susy";

int main(){
    print("Hello World!");
    print("Hello", name, ". Welcome to the Compilers course!");
    return;
}
```

A seguinte AST é produzida:

```
Program:
    GlobalDecl:
        Decl: ID(name=name)
            ArrayDecl:
                VarDecl:
                    Type: char @ 1:1
            Constant: string, "Susy" @ 1:15
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:11
            Print: @ 4:5
                Constant: string, "Hello World!" @ 4:11
            Print: @ 5:5
                ExprList: @ 5:11
                    Constant: string, "Hello" @ 5:11
                    ID: name @ 5:20
                    Constant: string, ". Welcome to the Compilers course!" @ 5:26
            Return: @ 6:5
```

## Esboço da Construção da AST

Complete o código abaixo, escrevendo classes de nós AST para cada construção da linguagem uC.

In [ ]:
import sys

def represent_node(obj, indent):
    def _repr(obj, indent, printed_set):
        """
        Get the representation of an object, with dedicated pprint-like format for lists.
        """
        if isinstance(obj, list):
            indent += 1
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            return (
                "["
                + (sep.join((_repr(e, indent, printed_set) for e in obj)))
                + final_sep
                + "]"
            )
        elif isinstance(obj, Node):
            if obj in printed_set:
                return ""
            else:
                printed_set.add(obj)
            result = obj.__class__.__name__ + "("
            indent += len(obj.__class__.__name__) + 1
            attrs = []
            for name in obj.__slots__:
                if name == "bind":
                    continue
                value = getattr(obj, name)
                value_str = _repr(value, indent + len(name) + 1, printed_set)
                attrs.append(name + "=" + value_str)
            sep = ",\n" + (" " * indent)
            final_sep = ",\n" + (" " * (indent - 1))
            result += sep.join(attrs)
            result += ")"
            return result
        elif isinstance(obj, str):
            return obj
        else:
            return ""

    # avoid infinite recursion with printed_set
    printed_set = set()
    return _repr(obj, indent, printed_set)

class Node:
    """Abstract base class for AST nodes."""

    __slots__ = ("coord",)

    def __init__(self, coord=None):
        self.coord = coord

    def children(self):
        """A sequence of all children that are Nodes"""
        pass

    attr_names = ()

    def __repr__(self):
        """Generates a python representation of the current node"""
        return represent_node(self, 0)

    def show(
        self,
        buf=sys.stdout,
        offset=0,
        attrnames=False,
        nodenames=False,
        showcoord=False,
        _my_node_name=None,
    ):
        """Pretty print the Node and all its attributes and children (recursively) to a buffer.
        buf:
            Open IO buffer into which the Node is printed.
        offset:
            Initial offset (amount of leading spaces)
        attrnames:
            True if you want to see the attribute names in name=value pairs. False to only see the values.
        nodenames:
            True if you want to see the actual node names within their parents.
        showcoord:
            Do you want the coordinates of each Node to be displayed.
        """
        lead = " " * offset
        if nodenames and _my_node_name is not None:
            buf.write(lead + self.__class__.__name__ + " <" + _my_node_name + ">: ")
            inner_offset = len(self.__class__.__name__ + " <" + _my_node_name + ">: ")
        else:
            buf.write(lead + self.__class__.__name__ + ":")
            inner_offset = len(self.__class__.__name__ + ":")

        if self.attr_names:
            if attrnames:
                nvlist = [
                    (n, represent_node(getattr(self, n), offset+inner_offset+1+len(n)+1))
                    for n in self.attr_names
                    if getattr(self, n) is not None
                ]
                attrstr = ", ".join("%s=%s" % nv for nv in nvlist)
            else:
                vlist = [getattr(self, n) for n in self.attr_names]
                attrstr = ", ".join(
                    represent_node(v, offset + inner_offset + 1) for v in vlist
                )
            buf.write(" " + attrstr)

        if showcoord:
            if self.coord and self.coord.line != 0:
                buf.write(" %s" % self.coord)
        buf.write("\n")

        for (child_name, child) in self.children():
            child.show(buf, offset + 4, attrnames, nodenames, showcoord, child_name)

class ArrayDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("type", "size",)

    def __init__(self, type, size, coord=None):
        super().__init__(coord)
        self.type = type
        self.size = size

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.size is not None:
            nodelist.append(('size', self.size))
        return tuple(nodelist)



class ArrayRef(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "subscript",)

    def __init__(self, name, subscript, coord=None):
        super().__init__(coord)
        self.name = name
        self.subscript = subscript

    def children(self):
        nodelist = []
        if self.name is not None:
            nodelist.append(('name', self.name))
        if self.subscript is not None:
            nodelist.append(('subscript', self.subscript))
        return tuple(nodelist)

class Assert(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Assignment(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("lvalue", "rvalue",)

    def __init__(self, lvalue, rvalue, coord=None):
        super().__init__(coord)
        self.lvalue = lvalue
        self.rvalue = rvalue

    def children(self):
        nodelist = []
        if self.lvalue is not None:
            nodelist.append(('lvalue', self.lvalue))
        if self.rvalue is not None:
            nodelist.append(('rvalue', self.rvalue))
        return tuple(nodelist)

class BinaryOp(Node):

    __slots__ = ("op", "left", "right",)

    def __init__(self, op, left, right, coord=None):
        super().__init__(coord)
        self.op = op
        self.left = left
        self.right = right

    def children(self):
        nodelist = []
        if self.left is not None:
            nodelist.append(('left', self.left))
        if self.right is not None:
            nodelist.append(('right', self.right))
        return tuple(nodelist)

    attr_names = ("op",)

class Break(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)
       

    def children(self):
        nodelist = []
        
        return tuple(nodelist)

    

class Cast(Node):   #eu add esse

    __slots__ = ("type", "expr",)

    def __init__(self, type, expr, coord=None):
        super().__init__(coord)
        self.type = type
        self.expr = expr

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)
class Compound(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("dcls", "stmts")

    def __init__(self, dcls, stmts, coord=None):
        super().__init__(coord)
        self.dcls = dcls
        self.stmts = stmts

    def children(self):
        nodelist = []
        for i, child in enumerate(self.dcls or []):
            nodelist.append(('dcls[%d]' % i, child))
        for i, child in enumerate(self.stmts or []):
            nodelist.append(('stmts[%d]' % i, child))    
        return tuple(nodelist)



class Constant(Node):

    __slots__ = ("type", "value",)

    def __init__(self, type, value, coord=None):
        super().__init__(coord)
        self.type = type
        self.value = value

    def children(self):
        return ()

    attr_names = ("type", "value",)

class Decl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "type", "init",)

    def __init__(self, name, type, init, coord=None):
        super().__init__(coord)
        self.name = name
        self.type = type
        self.init = init

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        if self.init is not None:
            nodelist.append(('init', self.init))
        return tuple(nodelist)

    attr_names = ("name",)

class DeclList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("decls", )

    def __init__(self, decls, coord=None):
        super().__init__(coord)
        self.decls = decls
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.decls or []):
            nodelist.append(('decls[%d]' % i, child))   
        return tuple(nodelist)
class EmptyStatement(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ()

    def __init__(self, coord=None):
        super().__init__(coord)
       

    def children(self):
        nodelist = []
        
        return tuple(nodelist)

class ExprList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("exprs", )

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))   
        return tuple(nodelist)

class For(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("init", "cond", "next", "statements",)

    def __init__(self, init, cond, next, statements, coord=None):
        super().__init__(coord)
        self.init = init
        self.cond = cond
        self.next = next
        self.statements = statements

    def children(self):
        nodelist = []
        if self.init is not None:
            nodelist.append(('init', self.init))
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.next is not None:
            nodelist.append(('next', self.next))    
        if self.statements is not None:
            nodelist.append(('statements', self.statements))    
        return tuple(nodelist)

class FuncCall(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name", "args",)

    def __init__(self, name, args, coord=None):
        super().__init__(coord)
        self.name = name
        self.args = args

    def children(self):
        nodelist = []
        if self.name is not None:
            nodelist.append(('name', self.name))
        if self.args is not None:
            nodelist.append(('args', self.args))
        return tuple(nodelist)

class FuncDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("args", "type",)

    def __init__(self, args, type, coord=None):
        super().__init__(coord)
        self.args = args
        self.type = type

    def children(self):
        nodelist = []
        if self.args is not None:
            nodelist.append(('args', self.args))
        if self.type is not None:
            nodelist.append(('type', self.type))
        return tuple(nodelist)

class FuncDef(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("spec", "decl","param_decls", "statements")

    def __init__(self, spec, decl, param_decls, statements, coord=None):
        super().__init__(coord)
        self.spec = spec
        self.decl = decl
        self.param_decls = param_decls
        self.statements = statements

    def children(self):
        nodelist = []
        if self.spec is not None:
            nodelist.append(('spec', self.spec))
        if self.decl is not None:
            nodelist.append(('decl', self.decl))
        for i, child in enumerate(self.param_decls or []):
            nodelist.append(('param_decls[%d]' % i, child))   
        if self.statements is not None:
            nodelist.append(('statements', self.statements))      
        return tuple(nodelist)

class GlobalDecl(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("decls", )

    def __init__(self, decls, coord=None):
        super().__init__(coord)
        self.decls = decls
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.decls or []):
            nodelist.append(('decls[%d]' % i, child))   
        return tuple(nodelist)

class ID(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name
        
    def children(self):
        nodelist = []
 
        return tuple(nodelist)

    attr_names = ("name",)


class If(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("cond", "if_statements", "else_statements",)

    def __init__(self, cond, if_statements, else_statements, coord=None):
        super().__init__(coord)
        self.cond = cond
        self.if_statements = if_statements
        self.else_statements = else_statements

    def children(self):
        nodelist = []
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.args is not None:
            nodelist.append(('if_statements', self.if_statements))
        if self.args is not None:
            nodelist.append(('else_statements', self.else_statements))
        return tuple(nodelist)

class InitList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("exprs", )

    def __init__(self, exprs, coord=None):
        super().__init__(coord)
        self.exprs = exprs
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.exprs or []):
            nodelist.append(('exprs[%d]' % i, child))   
        return tuple(nodelist)

class ParamList(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("params", )

    def __init__(self, params, coord=None):
        super().__init__(coord)
        self.params = params
        

    def children(self):
        nodelist = []
        for i, child in enumerate(self.params or []):
            nodelist.append(('params[%d]' % i, child))   
        return tuple(nodelist)


class Print(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Program(Node):

    __slots__ = ("gdecls",)

    def __init__(self, gdecls, coord=None):
        super().__init__(coord)
        self.gdecls = gdecls

    def children(self):
        nodelist = []
        for i, child in enumerate(self.gdecls or []):
            nodelist.append(('gdecls[%d]' % i, child))
        return tuple(nodelist)

class Read(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Return(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("expr",)

    def __init__(self, expr, coord=None):
        super().__init__(coord)
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        return tuple(nodelist)

class Type(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("name",)

    def __init__(self, name, coord=None):
        super().__init__(coord)
        self.name = name
        
    def children(self):
        nodelist = []
 
        return tuple(nodelist)

    attr_names = ("name",)


class VarDecl(Node):

    # <<< YOUR CODE HERE >>>
     # <<< YOUR CODE HERE >>>
    __slots__ = ("declname", "type",)

    def __init__(self, declname=None, type=None, coord=None):
        super().__init__(coord)
        self.declname = declname
        self.type = type

    def children(self):
        nodelist = []
        if self.type is not None:
            nodelist.append(('type', self.type))
        
        return tuple(nodelist)

   

class UnaryOp(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("op", "expr",)

    def __init__(self, op, expr, coord=None):
        super().__init__(coord)
        self.op = op
        self.expr = expr

    def children(self):
        nodelist = []
        if self.expr is not None:
            nodelist.append(('expr', self.expr))
        
        return tuple(nodelist)

    attr_names = ("op",)

class While(Node):

    # <<< YOUR CODE HERE >>>
    __slots__ = ("cond", "statements",)

    def __init__(self, cond, statements, coord=None):
        super().__init__(coord)
        self.cond = cond
        self.statements = statements

    def children(self):
        nodelist = []
        if self.cond is not None:
            nodelist.append(('cond', self.cond))
        if self.statements is not None:
            nodelist.append(('statements', self.statements))
        return tuple(nodelist)

In [ ]:
!pip install sly
from sly import Lexer, Parser

Copie o código da classe `UCLexer` que você escreveu no [primeiro projeto](https://colab.research.google.com/drive/1TFZwdVJGMVDo48cq63LsWGnqC9UXx3n1?usp=sharing) e cole na célula abaixo.

In [ ]:
class UCLexer(Lexer):
    """A lexer for the uC language."""

    def __init__(self, error_func):
        """Create a new Lexer.
        An error function. Will be called with an error
        message, line and column as arguments, in case of
        an error during lexing.
        """
        self.error_func = error_func
        
    # Reserved keywords
    keywords = {
        'assert': "ASSERT",
        'break': "BREAK",
        'char': "CHAR",
        'else': "ELSE",
        'for': "FOR",
        'if': "IF",
        'int': "INT",
        'print': "PRINT",
        'read': "READ",
        'return': "RETURN",
        'void': "VOID",
        'while': "WHILE",
    }

    # All the tokens recognized by the lexer
    tokens = tuple(keywords.values()) + (
         # Identifiers
        "ID",
        # constants
        "INT_CONST",
        "CHAR_CONST",
        "STRING_LITERAL",
        # Operators
        "PLUS",
        "MINUS",
        "TIMES",
        "DIVIDE",
        "MOD",
        "LE",
        "LT",
        "GE",
        "GT",
        "EQ",
        "NE",
        "OR",
        "AND",
        "NOT",
        # Assignment
        "EQUALS",
        # Delimeters
        "LPAREN",
        "RPAREN",  # ( )
        "LBRACKET",
        "RBRACKET",  # [ ]
        "LBRACE",
        "RBRACE",  # { }
        "COMMA",
        "SEMI",  # , ;
    )

    # String containing ignored characters (between tokens)
    ignore = " \t"

    # Other ignored patterns
    ignore_newline = r'[\\\n]'# <<< INCLUDE A REGEX HERE FOR NEWLINE >>>
    ignore_comment = r'(\/\*([\s\S]*?)\*\/)|(\/\/.*\n)'# <<< INCLUDE A REGEX HERE FOR COMMENT >>>

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][0-9a-zA-Z_]*'# <<< INCLUDE A REGEX HERE FOR ID >>>
    INT_CONST = r'[0-9]+'# <<< INCLUDE A REGEX HERE FOR INT_CONST >>>
    CHAR_CONST = r'(\'(\\.|[^\\]?)\')' # <<< INCLUDE A REGEX HERE FOR CHAR_CONST >>>
    # <<< YOUR CODE HERE >>>
    STRING_LITERAL = r'(\"(.|(\\\"))*?[^\\]\")'
    PLUS = r'\+'
    MINUS = r'\-'
    TIMES = r'\*'
    DIVIDE = r'\/(?!\*)'
    MOD = r'\%'
    LE = r'\<\='
    LT = r'\<(?!\=)'
    GE = r'\>\='
    GT = r'\>(?!\=)'
    EQ = r'\=\='
    NE = r'\!\='
    OR = r'\|\|'
    AND = r'\&\&'
    NOT = r'\!(?!\=)'
    EQUALS = r'\=(?!\=)'
        # Delimeters
    LPAREN = r'\('
    RPAREN = r'\)'
    LBRACKET = r'\['
    RBRACKET = r'\]'
    LBRACE = r'\{'
    RBRACE = r'\}'
    COMMA = r'\,'
    SEMI = r'\;'

    #erros
    unmatched_quote = r'\"[^\"]*?(?=\n)'

    def unmatched_quote(self, t):
      msg = "Unterminated string literal"
      self._error(msg, t)

    uccomment = r'(\/\*)([\s\S]*?)[^(\*\/)]\Z'

    def uccomment(self, t):
      msg = "Unterminated comment"
      self._error(msg, t)

    Unterminated_character = r'\'[^\']*?(?=\n)|\'.{2,}\'|\'\\\''

    def Unterminated_character(self, t):
      msg = "Unterminated character const"
      self._error(msg, t)

    # Special cases
    def ID(self, t):
      t.type = self.keywords.get(t.value, "ID")
      return t

    # Define a rule so we can track line numbers
    def ignore_newline(self, t):
      self.lineno += len(t.value)
 
    def ignore_comment(self, t):
      self.lineno += t.value.count("\n")

    def find_tok_column(self, token):
        """Find the column of the token in its line."""
        last_cr = self.text.rfind('\n', 0, token.index)
        if last_cr < 0: last_cr = 0
        return token.index - last_cr + 1

    # Internal auxiliary methods
    def _error(self, msg, token):
        location = self._make_location(token)
        self.error_func(msg, location[0], location[1])
        self.index += 1

    def _make_location(self, token):
        return token.lineno, self.find_tok_column(token)

    # Error handling rule
    def error(self, t):
        msg = "Illegal character %s" % repr(t.value[0])
        self._error(msg, t)

    # Scanner (used only for test)
    def scan(self, text):
        output = ""
        for tok in self.tokenize(text):
            print(tok)
            output += str(tok) + "\n"
        return output

Use o código da classe `UCParser` que você escreveu no [segundo projeto](https://colab.research.google.com/drive/1YOIaXvrFIzZqLqNa-VHIvJq7tQkzqTbu?usp=sharing) como base para completar completar o código abaixo:
- O código abaixo inclui funções auxiliares para corrigir a construção de declarações e lidar com modificador de tipo.
- Defina as regras de precedência e associatividade conforme a implementação que você escreveu no [segundo projeto](https://colab.research.google.com/drive/1YOIaXvrFIzZqLqNa-VHIvJq7tQkzqTbu?usp=sharing). 
- Complete o código dos métodos referentes às regras gramaticas usando a implementação do [segundo projeto](https://colab.research.google.com/drive/1YOIaXvrFIzZqLqNa-VHIvJq7tQkzqTbu?usp=sharing).
- Modifique as regras para retornarem nós AST ao invés de tuplas e criando nós AST para identificadores e constantes.
- Use as funções auxiliares para corrigir a construção de declarações e lidar com modificador de tipo.

In [ ]:
class UCParser(Parser):
    """A parser for the uC language."""

    # Get the token list from the lexer (required)
    tokens = UCLexer.tokens

    precedence = (
        # <<< YOUR CODE HERE >>>
        ('left', 'ELSE'), 
        ('left', 'FOR'),
        ('left', 'WHILE'),  
        ('left', 'BREAK'),
        ('left', 'READ', 'PRINT'),
        ('left', 'RPAREN', 'LPAREN', 'LBRACKET', 'RBRACKET'),
        ('left', 'LBRACE'),
        ('left', 'RBRACE'),
        ('right', 'NOT'),
        ('left', 'AND', 'OR'),
        ('left', 'EQ', 'NE'),
        ('nonassoc', 'LE', 'LT', 'GE', 'GT'),
        ('left', 'PLUS', 'MINUS'),
        ('left', 'DIVIDE','TIMES', 'MOD'),
        ('right', 'EQUALS')
    )
    
    def __init__(self, error_func=lambda msg, x, y: print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)):
        """Create a new Parser.
        An error function for the lexer.
        """
        self.lexer = UCLexer(error_func)

    def parse(self, text, lineno=1, index=0):
        return super().parse(self.lexer.tokenize(text, lineno, index))

    # Internal auxiliary methods
    def _token_coord(self, p):
        line, column = self.lexer._make_location(p)
        return Coord(line, column)

    def _build_declarations(self, spec, decls):
        """ Builds a list of declarations all sharing the given specifiers.
        """
        declarations = []

        for decl in decls:
            assert decl['decl'] is not None
            declaration = Decl(
                    name=None,
                    type=decl['decl'],
                    init=decl.get('init'),
                    coord=decl['decl'].coord)

            fixed_decl = self._fix_decl_name_type(declaration, spec)
            declarations.append(fixed_decl)

        return declarations

    def _fix_decl_name_type(self, decl, typename):
        """ Fixes a declaration. Modifies decl.
        """
        # Reach the underlying basic type
        type = decl
        while not isinstance(type, VarDecl):
            type = type.type

        decl.name = type.declname
        if not typename:
            # Functions default to returning int
            if not isinstance(decl.type, FuncDecl):
                self._parse_error("Missing type in declaration", decl.coord)
            type.type = Type('int', coord=decl.coord)
        else:
            type.type = Type(typename.name, coord=typename.coord)
            
        return decl
        
    def _type_modify_decl(self, decl, modifier):
        """Tacks a type modifier on a declarator, and returns
           the modified declarator.
           Note: the declarator and modifier may be modified
        """
        modifier_head = modifier
        modifier_tail = modifier

        # The modifier may be a nested list. Reach its tail.
        while modifier_tail.type:
            modifier_tail = modifier_tail.type

        # If the decl is a basic type, just tack the modifier onto it
        if isinstance(decl, VarDecl):
            modifier_tail.type = decl
            return modifier
        else:
            # Otherwise, the decl is a list of modifiers. Reach
            # its tail and splice the modifier onto the tail,
            # pointing to the underlying basic type.
            decl_tail = decl

            while not isinstance(decl_tail.type, VarDecl):
                decl_tail = decl_tail.type

            modifier_tail.type = decl_tail.type
            decl_tail.type = modifier_head
            return decl

    # Error handling rule
    def error(self, p):
        if p:
            if hasattr(p, 'lineno'):
                print("Error at line %d near the symbol %s " % (p.lineno, p.value))
            else:
                print("Error near the symbol %s" % p.value)
        else:
            print("Error at the end of input")

    # <program> ::= {<global_declaration>}+
    @_('global_declaration_list')
    def program(self, p):
        return Program(p.global_declaration_list)
        
    @_('global_declaration { global_declaration }')
    def global_declaration_list(self, p):
        return [p.global_declaration0] + p.global_declaration1        

    # <global_declaration> ::= <function_definition>
    #                        | <declaration>
    @_('function_definition',
       'declaration')
    def global_declaration(self, p):
        if hasattr(p, 'function_definition'):
            return p.function_definition
        else:
            return GlobalDecl(p.declaration)

    # <function_definition> ::= <type_specifier> <declarator> {<declaration>}* <compound_statement>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator { declaration } compound_statement' )
    def function_definition(self, p):
        return (FuncDef(p.type_specifier, self._build_declarations(spec=p.type_specifier, decls=[dict(decl= p.declarator, init=None)])[0], p.declaration, p.compound_statement))  
    # <type_specifier> ::= "void"
    #                    | "char"
    #                    | "int"
    # <<< YOUR CODE HERE >>>
    @_('VOID')
    def type_specifier(self, p):
        return (Type('void', coord=self._token_coord(p)))

    @_('CHAR')
    def type_specifier(self, p):
       return (Type('char', coord=self._token_coord(p))) 

    @_('INT')
    def type_specifier(self, p):
        return (Type('int', coord=self._token_coord(p)))    

    # <declarator> ::= <identifier>
    #                | "(" <declarator> ")"
    #                | <declarator> "[" {<constant_expression>}? "]"
    #                | <declarator> "(" {<parameter_list>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('ID')
    def declarator(self, p):
        return (VarDecl(ID(str(p.ID), coord= self._token_coord(p))))  #faltou testar. Não temos 100% de certeza aqui

    @_('LPAREN declarator RPAREN')
    def declarator(self, p):
        return (p.declarator) 

    @_('declarator LBRACKET [ constant_expression ] RBRACKET')
    def declarator(self, p):
        return (self._type_modify_decl(decl= p.declarator, modifier = ArrayDecl(None, p.constant_expression, coord= p.declarator.coord(p))))    

    @_('declarator LPAREN [ parameter_list ] RPAREN')
    def declarator(self, p):
        return (self._type_modify_decl(decl= p.declarator, modifier = FuncDecl(p.parameter_list, None, coord= p.declarator.coord(p))))  

   
    # <constant_expression> ::= <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def constant_expression(self, p):
        return (p.binary_expression)

    # <binary_expression> ::= <unary_expression>
    #                       | <binary_expression>  "*"   <binary_expression>
    #                       | <binary_expression>  "/"   <binary_expression>
    #                       | <binary_expression>  "%"   <binary_expression>
    #                       | <binary_expression>  "+"   <binary_expression>
    #                       | <binary_expression>  "-"   <binary_expression>
    #                       | <binary_expression>  "<"   <binary_expression>
    #                       | <binary_expression>  "<="  <binary_expression>
    #                       | <binary_expression>  ">"   <binary_expression>
    #                       | <binary_expression>  ">="  <binary_expression>
    #                       | <binary_expression>  "=="  <binary_expression>
    #                       | <binary_expression>  "!="  <binary_expression>
    #                       | <binary_expression>  "&&"  <binary_expression>
    #                       | <binary_expression>  "||"  <binary_expression>
    # <<< YOUR CODE HERE >>>
    @_('unary_expression')
    def binary_expression(self, p):
        return (p.unary_expression)

    @_('binary_expression TIMES binary_expression', 
       'binary_expression DIVIDE binary_expression', 
       'binary_expression MOD binary_expression',
       'binary_expression PLUS binary_expression',
       'binary_expression MINUS binary_expression', 
       'binary_expression LT binary_expression',
       'binary_expression LE binary_expression',
       'binary_expression GT binary_expression',
       'binary_expression GE binary_expression',
       'binary_expression EQ binary_expression',
       'binary_expression NE binary_expression',
       'binary_expression AND binary_expression',
       'binary_expression OR binary_expression')
    def binary_expression(self, p):
        return (BinaryOp(p[1], p.binary_expression0, p.binary_expression1, coord= self._token_coord(p)))   #testar isso aqui tambem

    # <unary_expression> ::= <postfix_expression>
    #                      | <unary_operator> <unary_expression>
    # <<< YOUR CODE HERE >>>
    @_('postfix_expression')
    def unary_expression(self, p):
        return (p.postfix_expression)

    @_('unary_operator unary_expression')
    def unary_expression(self, p):
        return (UnaryOp(p.unary_operator, p.unary_expression, coord= p.unary_expression.coord(p)))   

    # <postfix_expression> ::= <primary_expression>
    #                        | <postfix_expression> "[" <expression> "]"
    #                        | <postfix_expression> "(" {<argument_expression>}? ")"
    # <<< YOUR CODE HERE >>>
    @_('primary_expression')
    def postfix_expression(self, p):
        return (p.primary_expression)

    @_('postfix_expression LBRACKET expression RBRACKET')
    def postfix_expression(self, p):
        return (ArrayRef(p.postfix_expression, p.expression, coord= p.postfix_expression.coord(p)))   

    @_('postfix_expression LPAREN [ argument_expression ] RPAREN ')
    def postfix_expression(self, p):
        return (FuncCall(p.postfix_expression, p.argument_expression, coord=p.postfix_expression.coord(p)))  

    # <primary_expression> ::= <identifier>
    #                        | <constant>
    #                        | <string>
    #                        | "(" <expression> ")"
    # <<< YOUR CODE HERE >>>
    @_('ID')
    def primary_expression(self, p):
        return (ID(str(p.ID), coord=self._token_coord(p))) #conferir depois
    
    @_('constant')
    def primary_expression(self, p):
        return (p.constant)   
    
    @_('STRING_LITERAL')
    def primary_expression(self, p):
        return (Constant('string', str(p.STRING_LITERAL), coord=self._token_coord(p)))  

    @_('LPAREN expression RPAREN')
    def primary_expression(self, p):
        return (p.expression)

    # <constant> ::= <integer_constant>
    #              | <character_constant>
    # <<< YOUR CODE HERE >>>
    @_('INT_CONST')
    def constant(self, p):
        return (Constant('int', str(p.INT_CONST), coord=self._token_coord(p)))
   
    @_('CHAR_CONST')
    def constant(self, p):
        return (Constant('char', str(p.CHAR_CONST), coord=self._token_coord(p)))    

    # <expression> ::= <assignment_expression>
    #                | <expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def expression(self, p):
        return ( p.assignment_expression)
    @_('expression COMMA assignment_expression ')
    def expression(self, p):
        return(ExprList([p.expression] + p.assignment_expression, coord=p.assignment_expression.coord(p)))
    # <argument_expression> ::= <assignment_expression>
    #                         | <argument_expression> "," <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def argument_expression(self, p):
        return ( p.assignment_expression)
    @_('argument_expression COMMA assignment_expression')
    def argument_expression(self, p):
        return(ExprList([p.argument_expression] + p.assignment_expression, coord=p.assignment_expression.coord(p)))  

    # <assignment_expression> ::= <binary_expression>
    #                           | <unary_expression> "=" <assignment_expression>
    # <<< YOUR CODE HERE >>>
    @_('binary_expression')
    def assignment_expression(self, p):
        return (p.binary_expression)

    @_('unary_expression EQUALS assignment_expression')
    def assignment_expression(self, p):
        return (Assignment(p.unary_expression, p.assignment_expression, coord=p.unary_expression.coord(p)))

    # <unary_operator> ::= "+"
    #                    | "-"
    #                    | "!"
    # <<< YOUR CODE HERE >>>
    @_('PLUS')
    def unary_operator(self, p):
        return ('+') 

    @_('MINUS')
    def unary_operator(self, p):
        return ('-')     

    @_('NOT')
    def unary_operator(self, p):
        return ('!') 
    # <parameter_list> ::= <parameter_declaration>
    #                    | <parameter_list> "," <parameter_declaration>
    # <<< YOUR CODE HERE >>>
   # @_('')   #checar
   # def parameter_list(self, p):
   #     return (p.parameter_list)

    @_('parameter_declaration { COMMA parameter_declaration }')
    def parameter_list(self, p):
        return (ParamList([p.parameter_declaration0] + p.parameter_declaration1 , coord=p.parameter_declaration.coord(p))
)   

    # <parameter_declaration> ::= <type_specifier> <declarator>
    # <<< YOUR CODE HERE >>>
    @_('type_specifier declarator')
    def parameter_declaration(self, p):
        return (self._build_declarations(spec=p.type_specifier, decls=[dict(decl=p.declarator, init=None)])[0])

    # <declaration> ::=  <type_specifier> {<init_declarator_list>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('type_specifier [ init_declarator_list ] SEMI')
    def declaration(self, p):
        return (self._build_declarations(spec=p.type_specifier, decls=p.init_declarator_list))

    # <init_declarator_list> ::= <init_declarator>
    #                          | <init_declarator_list> "," <init_declarator>
    # <<< YOUR CODE HERE >>>
    #@_('init_declarator_list')                 #eu comentei pra testar
    #def init_declarator_list(self, p):
    #    return (list(p.init_declarator))

    @_('init_declarator { COMMA init_declarator }')
    def init_declarator_list(self, p):
        return ([ p.init_declarator0 ]+ p.init_declarator1)  

    # <init_declarator> ::= <declarator>
    #                     | <declarator> "=" <initializer>
    # <<< YOUR CODE HERE >>>
   
    @_('declarator [ EQUALS initializer ]')
    def init_declarator(self, p):
        return (dict(decl=p.declarator, init=p.initializer))

    # <initializer> ::= <assignment_expression>
    #                 | "{" {<initializer_list>}? "}"
    #                 | "{" <initializer_list> , "}"
    # <<< YOUR CODE HERE >>>
    @_('assignment_expression')
    def initializer(self, p):
        return (p.assignment_expression)
    @_('LBRACE [ initializer_list ] RBRACE')
    def initializer(self, p):
        return (p.initializer_list)   
    @_('LBRACE initializer_list "," RBRACE')
    def initializer(self, p):
        return (p.initializer_list)     

    # <initializer_list> ::= <initializer>
    #                      | <initializer_list> "," <initializer>
    # <<< YOUR CODE HERE >>>
   # @_('initializer_list')
    #def initializer_list(self, p):
     #   return (p.initializer_list) 

    @_('initializer {  COMMA initializer }')        #eu comentei
    def initializer_list(self, p):
        return ([ p.initializer0 ] + p.initializer1)     

    # <compound_statement> ::= "{" {<declaration>}* {<statement>}* "}"
    # <<< YOUR CODE HERE >>>
    @_('LBRACE { declaration } { statement } RBRACE')
    def compound_statement(self, p):
        return (Compound((dcflat for sublist in p.declaration for dcflat in sublist), p.statement, self._token_coord(p))) 
    # <statement> ::= <expression_statement>
    #               | <compound_statement>
    #               | <selection_statement>
    #               | <iteration_statement>
    #               | <jump_statement>
    #               | <assert_statement>
    #               | <print_statement>
    #               | <read_statement>
    # <<< YOUR CODE HERE >>>
    @_('expression_statement', 'compound_statement', 'selection_statement', 'iteration_statement', 'jump_statement', 'assert_statement', 'print_statement', 'read_statement')
    def statement(self, p):
        return (p[0]) 
    # <expression_statement> ::= {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('[ expression ] SEMI')
    def expression_statement(self, p):
        return (p.expression if p.expression is not None else EmptyStatement(coord=self._token_coord(p))) 

    # <selection_statement> ::= "if" "(" <expression> ")" <statement>
    #                         | "if" "(" <expression> ")" <statement> "else" <statement>
    # <<< YOUR CODE HERE >>>
    @_('IF LPAREN expression RPAREN statement [ ELSE statement ]')
    def selection_statement(self, p):
        return (If(p.expression, p.statement0, p.statement1, coord=self._token_coord(p))) 

    # <iteration_statement> ::= "while" "(" <expression> ")" <statement>
    #                         | "for" "(" {<expression>}? ";" {<expression>}? ";" {<expression>}? ")" <statement>
    #                         | "for" "(" <declaration> {<expression>}? ";" {<expression>}? ")" <statement>
    # <<< YOUR CODE HERE >>>
    @_('WHILE LPAREN expression RPAREN statement')
    def iteration_statement(self, p):
        return (While(p.expression, p.statement, coord=self._token_coord(p)))

    @_('FOR LPAREN [ expression ] SEMI [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return (For(p.expression0, p.expression1, p.expression2, p.statement, coord=self._token_coord(p)))

    @_('FOR LPAREN declaration [ expression ] SEMI [ expression ] RPAREN statement')
    def iteration_statement(self, p):
        return (For(DeclList(p.declaration, coord=self._token_coord(p)), p.expression0, p.expression1, p.statement, coord=self._token_coord(p)))    

    # <jump_statement> ::= "break" ";"
    #                    | "return" {<expression>}? ";"
    # <<< YOUR CODE HERE >>>
    @_('BREAK SEMI')
    def jump_statement(self, p):
        return (Break(coord=self._token_coord(p)))

    @_('RETURN [ expression ] SEMI')
    def jump_statement(self, p):
        return (Return(p.expression, coord=self._token_coord(p)))

    # <assert_statement> ::= "assert" <expression> ";"
    # <<< YOUR CODE HERE >>>
    @_('ASSERT expression SEMI')
    def assert_statement(self, p):
        return (Assert(p.expression, coord=self._token_coord(p)))

    # <print_statement> ::= "print" "(" {<expression>}? ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('PRINT LPAREN [ expression ] RPAREN SEMI')
    def print_statement(self, p):
        return (Print(p.expression, coord=self._token_coord(p)))

    # <read_statement> ::= "read" "(" <argument_expression> ")" ";"
    # <<< YOUR CODE HERE >>>
    @_('READ LPAREN argument_expression RPAREN SEMI')
    def read_statement(self, p):
        return (Read(p.argument_expression, coord=self._token_coord(p)))

In [ ]:
def print_error(msg, x, y):
    # use stdout to match with the output in the .out test files
    print("Lexical error: %s at %d:%d" % (msg, x, y), file=sys.stdout)
    
def main(args):
    parser = UCParser(print_error)
    with open(args[0], 'r') if len(args) > 0 else sys.stdin as f:
        ast = parser.parse(f.read())
        if ast is not None:
            ast.show(showcoord=True)

## Teste
Para o desenvolvimento inicial, tente executar o analisador sintático em um arquivo de entrada de exemplo, como:

```cpp
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}
```

In [ ]:
%%file test.uc 
/* comment */
int j = 3;
int main () {
  int i = j;
  int k = 3;
  int p = 2 * j;
  assert p == 2 * i;
}

E o resultado será semelhante ao texto mostrado abaixo.

```
Program:
    GlobalDecl:
        Decl: ID(name=j)
            VarDecl:
                Type: int @ 2:1
            Constant: int, 3 @ 2:9
    FuncDef:
        Type: int @ 3:1
        Decl: ID(name=main)
            FuncDecl:
                VarDecl:
                    Type: int @ 3:1
        Compound: @ 3:13
            Decl: ID(name=i)
                VarDecl:
                    Type: int @ 4:3
                ID: j @ 4:11
            Decl: ID(name=k)
                VarDecl:
                    Type: int @ 5:3
                Constant: int, 3 @ 5:11
            Decl: ID(name=p)
                VarDecl:
                    Type: int @ 6:3
                BinaryOp: * @ 6:11
                    Constant: int, 2 @ 6:11
                    ID: j @ 6:15
            Assert: @ 7:3
                BinaryOp: == @ 7:10
                    ID: p @ 7:10
                    BinaryOp: * @ 7:15
                        Constant: int, 2 @ 7:15
                        ID: i @ 7:19
```

In [ ]:
main(["test.uc"])

Estude cuidadosamente a representação textual da AST e certifique-se de que ela faz sentido. Quando estiver razoavelmente satisfeito com a saída, tente executar alguns dos testes mais complicados projetados para testar vários cenários atípicos, fora do padrão esperado. Você pode usar como base os exemplos contidos [aqui](https://colab.research.google.com/drive/13o4SdAWIlpDYZ6rU308LqiwXsd0dwRe5?usp=sharing). 

No [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=527882) há um grande conjunto de testes para verificar seu código: confira-os para ver mais exemplos.

## Envie seu trabalho

Depois de concluir esta tarefa, salve o código das [classes de nós AST](#scrollTo=okQJM6uCeBnv) em um arquivo chamado `uc_ast.py`, copie o código da [classe `UCParser`](#scrollTo=wjdOC2uKiRld) e os submeta no [AVA](https://ava2.ead.ufscar.br/mod/quiz/view.php?id=527882), carregando o arquivo `uc_ast.py` e colando o código da classe `UCParser`.

## Anexo

A lista abaixo define os campos que devem ser armazenados em cada tipo de nó AST:

```
###########################################################
## Nós do uC ##############################################
###########################################################

#   <name>*     - um nó filho
#   <name>**    - uma sequência de nós filhos
#   <name>      - um atributo

ArrayDecl: [type*, size*]

ArrayRef: [name*, subscript*]

Assert: [expr*]

Assignment: [lvalue*, rvalue*]

BinaryOp: [op, left*, right*]

Break: []

Cast: [type*, expr*]

Compound: [decls**, stmts**]

Constant: [type, value]

Decl: [name, type*, init*]

DeclList: [decls**]

EmptyStatement: []

ExprList: [exprs**]

For: [init*, cond*, next*, statements*]

FuncCall: [name*, args*]

FuncDecl: [args*, type*]

FuncDef: [spec*, decl*, param_decls**, statements*]

GlobalDecl: [decls**]

ID: [name]

If: [cond*, if_statements*, else_statements*]

InitList: [exprs**]

ParamList: [params**]

Print: [expr*]

Program: [gdecls**] # OBS: pode ser GlobalDecl ou FuncDef

Read: [expr*]

Return: [expr*]

Type: [name]

VarDecl: [declname, type*] # OBS: não imprima declname

UnaryOp: [op, expr*]

While: [cond*, statements*]
```

A lista abaixo define os nós AST que devem ser retornados em cada regra da gramática:

```
program = Program(list(global_declaration))

global_declaration = function_definition 
                   | GlobalDecl(declaration)

function_definition = FuncDef(type_specifier, build_declarations(spec=type_specifier, decls=[dict(decl=declarator, init=None)])[0], declaration, compound_statement)

type_specifier = Type('void', coord=(lineno, column))
               | Type('char', coord=(lineno, column))
               | Type('int', coord=(lineno, column))

declarator = VarDecl(ID(str(ID), coord=(lineno, column)))
           | declarator
           | type_modify_decl(decl=declarator, modifier=ArrayDecl(None, constant_expression, coord=declarator.coord))
           | type_modify_decl(decl=declarator, modifier=FuncDecl(parameter_list, None, coord=declarator.coord))

constant_expression = binary_expression

binary_expression = unary_expression
                  | BinaryOp('*', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('/', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('%', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('+', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('-', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('<', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('<=', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('>', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('>=', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('==', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('!=', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('&&', binary_expression0, binary_expression1, coord=binary_expression0.coord)
                  | BinaryOp('||', binary_expression0, binary_expression1, coord=binary_expression0.coord)

unary_expression = postfix_expression
                 | UnaryOp(unary_operator, unary_expression, coord=unary_expression.coord)

postfix_expression = primary_expression
                   | ArrayRef(postfix_expression, expression, coord=postfix_expression.coord)
                   | FuncCall(postfix_expression, argument_expression, coord=postfix_expression.coord)

primary_expression = ID(str(ID), coord=(lineno, column))
                   | constant
                   | Constant('string', str(STRING_LITERAL), coord=(lineno, column))
                   | expression

constant = Constant('int', str(INT_CONST), coord=(lineno, column))
         | Constant('char', str(CHAR_CONST), coord=(lineno, column))

expression = assignment_expression
           | ExprList(list(assignment_expression), coord=assignment_expression.coord)
               
argument_expression = assignment_expression
                    | ExprList(list(assignment_expression), coord=assignment_expression.coord)

assignment_expression = binary_expression
                      | Assignment(unary_expression, assignment_expression, coord=unary_expression.coord)

unary_operator = '+'
               | '-'
               | '!'

parameter_list = ParamList(list(parameter_declaration), coord=parameter_declaration.coord)

parameter_declaration = build_declarations(spec=type_specifier, decls=[dict(decl=declarator, init=None)])[0]

declaration = build_declarations(spec=type_specifier, decls=init_declarator_list)

init_declarator_list = list(init_declarator)

init_declarator = dict(decl=declarator, init=initializer)

initializer = assignment_expression
            | initializer_list
            | initializer_list

initializer_list = InitList(list(initializer), coord=initializer.coord)

compound_statement = Compound(flatten(declaration), statement)

statement = expression_statement
          | compound_statement
          | selection_statement
          | iteration_statement
          | jump_statement
          | assert_statement
          | print_statement
          | read_statement

expression_statement = expression if expression is not None else EmptyStatement(coord=(lineno, column))

selection_statement = If(expression, statement0, statement1, coord=(lineno, column))

iteration_statement = While(expression, statement, coord=(lineno, column))
                    | For(expression0, expression1, expression2, statement, coord=(lineno, column))
                    | For(DeclList(declaration, coord=(lineno, column)), expression0, expression1, statement, coord=(lineno, column))

jump_statement = Break(coord=(lineno, column))
               | Return(expression, coord=(lineno, column))

assert_statement = Assert(expression, coord=(lineno, column))

print_statement = Print(expression, coord=(lineno, column))

read_statement = Read(argument_expression, coord=(lineno, column))
```
